### קוד מבוא

In [1]:
import pandas as pd
import os
import sys

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

In [3]:
cwd = os.getcwd()

create_forecast_basic_folder_path = os.path.dirname(cwd)

sys.path.append(create_forecast_basic_folder_path)

### פונקציות

In [4]:
from global_functions import get_forecast_version_folder_location, get_model_versions_folder_location, get_newest_date_file, up_load_df, sum_scenarios, group_sector, compare_scenarios

In [5]:
cbs_machoz_past_data=up_load_df(r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\comparison_tools\background_files','cbs_machoz_past_data')

In [6]:
jtmt=up_load_df(r'W:\Data\Forecast\forecast_by_version\V4\OVERVIEW','240701_forecast_2020_till_2050_jtmt')

In [7]:
iplan=up_load_df(r'W:\Data\Forecast\forecast_by_version\V4\OVERVIEW','240701_forecast_2020_till_2050_iplan')

In [8]:
bau=up_load_df(r'W:\Data\Forecast\forecast_by_version\V4\OVERVIEW','240701_forecast_2020_till_2050_bau')

In [9]:
col=['pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
  'pop_without_dorms_yeshiva_2050']

In [10]:
years = list(range(2025, 2051, 5))
years.insert(0,'zonetype')

In [11]:
def add_forecast_to_past(df,col,years,cbs_machoz_past_data,name):
    df=df.pivot_table(index='zonetype',aggfunc=sum)[col].reset_index()
    df.columns=years
    df=pd.merge(cbs_machoz_past_data,df,on='zonetype')
    df['soruce']=name
    return df

In [12]:
jtmt=add_forecast_to_past(jtmt,col,years,cbs_machoz_past_data,'JTMT')

In [13]:
iplan=add_forecast_to_past(iplan,col,years,cbs_machoz_past_data,'iplan')

In [14]:
bau=add_forecast_to_past(bau,col,years,cbs_machoz_past_data,'bau')

In [20]:
pd.concat([jtmt,iplan,bau]).to_excel(r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\comparison_tools\results\scenario_with_past.xlsx')

# ארכיון

### העלת קבצים

In [17]:
forecast_version_folder_location=get_forecast_version_folder_location(r'{}\background_files\forecast_version_folder_location.txt'.format(create_forecast_basic_folder_path))

forecast_version_base_year_folder_location=r'{}\BASE_YEAR'.format(forecast_version_folder_location)

In [18]:
forecast_version = get_newest_date_file(fr'{forecast_version_folder_location}', '_forecast_2020_till_2050_jtmt')

In [19]:
jtmt=up_load_df(forecast_version_folder_location, fr"{forecast_version}_forecast_2020_till_2050_jtmt")

jtmt=jtmt.query('zonetype!="Palestinian"')

FileNotFoundError: [Errno 2] No such file or directory: 'W:\\Data\\Forecast\\forecast_by_version\\V4\\240701_forecast_2020_till_2050_jtmt.xlsx'

In [8]:
bau=up_load_df(forecast_version_folder_location, fr"{forecast_version}_forecast_2020_till_2050_bau")

#bau=bau.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

In [9]:
iplan=up_load_df(forecast_version_folder_location, fr"{forecast_version}_forecast_2020_till_2050_iplan")

#iplan=iplan.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

In [10]:
model_versions_folder_location=get_model_versions_folder_location(r'{}\background_files\model_versions_folder_location.txt'.format(create_forecast_basic_folder_path))

In [11]:
v3=up_load_df(model_versions_folder_location,'v3_for_comparison_to_v4')

v3=v3.query('zonetype!="Palestinian"')

In [12]:
save_file=r'{}\background_files'.format(cwd)

past_2010=up_load_df(save_file,'נתוני ישובים ומועצות  2010 - עיבוד')

past_2000=up_load_df(save_file,'נתוני ישובים ומועצות  2000 -עיבוד')

past_2020=up_load_df(save_file,'נתוני רשויות 2020 - עיבוד')

gov=up_load_df(save_file,'טיוטת תוצאות לתרחישי אוכלוסייה אזוריים - 2020-2050 - טיוטה')

## אחוז תרחיש מכלל ארצי

In [13]:
def gov_goal_by_year(sum_df,df,year,scenario_True):
    goal=df.query('proj=="medium" & housing_adjusted ==@scenario_True & year==@year')['pop'].sum().item()
    
    data = [ {'scenario': 'iplan' if scenario_True else 'bau', 'year': year,'pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df

In [14]:
def gov_goal_by_year_machoz(sum_df,df,year,scenario_True,machoz_name):
    goal=df.query('proj=="medium" & housing_adjusted ==@scenario_True & year==@year & district==@machoz_name')['pop'].sum().item()
    
    data = [ {'scenario': 'iplan' if scenario_True else 'bau', 'year': year,'zonetype':machoz_name,'pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df    

In [15]:
def jtmt_forecast_by_year_machoz(sum_df,jtmt,year,machoz_name):
    goal=jtmt.query('zonetype==@machoz_name')['pop_without_dorms_yeshiva_{}'.format(year)].sum().item()
    
    data = [ {'scenario': 'jtmt' , 'year': year,'zonetype':machoz_name,'pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df

In [16]:
def jtmt_forecast_by_year_metro(sum_df,jtmt,year,metro):
    goal=jtmt.query('jeru_metro==@metro')['pop_without_dorms_yeshiva_{}'.format(year)].sum().item()
    
    data = [ {'scenario': 'jtmt' , 'year': year,'zonetype':'metro','pop':goal}]
     
    df_to_add = pd.DataFrame(data)
    
    sum_df=pd.concat([sum_df,df_to_add],axis=0)
    
    return sum_df

In [17]:
sum_df = pd.DataFrame()
for t in [True,False]:
    for m in ["Jerusalem District","Yehuda and Shomron"]:
        for y in list(range(2020,2051,5)):
            sum_df=gov_goal_by_year_machoz(sum_df,gov,y,t,m)

In [18]:
for m in ["Jerusalem","Judea and Samaria"]:
        for y in list(range(2025,2051,5)):
            sum_df=jtmt_forecast_by_year_machoz(sum_df,jtmt,y,m)

In [19]:

for y in list(range(2025,2051,5)):
    sum_df=jtmt_forecast_by_year_metro(sum_df,jtmt,y,1)

In [20]:
sum_df.loc[sum_df['zonetype']=="Jerusalem",'zonetype']="Jerusalem District"

In [21]:
sum_df.loc[sum_df['zonetype']=="Judea and Samaria",'zonetype']="Yehuda and Shomron"

In [ ]:
df=sum_df.pivot_table(index=['year'],columns=['zonetype','scenario'],aggfunc=sum)

In [ ]:
sum_total = pd.DataFrame()
for t in [True,False]:
    for y in list(range(2020,2051,5)):
        sum_total=gov_goal_by_year(sum_total,gov,y,t)

sum_total=sum_total.pivot_table(index=['year'],columns=['scenario'],aggfunc=sum)

In [24]:
df['Totatl_pop_il']=sum_total['pop']['bau']

In [25]:
df.to_excel('scenario_precent_of_pop_il.xlsx')

## נתוני עבר

In [ ]:
sum_df = pd.DataFrame()

col_index=['zonetype','muni_name']

col_need=[ 'pop_without_dorms_yeshiva','pop_0_20','pop_21_45','pop_45_65','pop_65+','student','jew_precent']

sum_df=sum_scenarios(sum_df,past_2000,'2000',col_index,col_need,2)

sum_df=sum_scenarios(sum_df,past_2010,'2010',col_index,col_need,2)

sum_df=sum_scenarios(sum_df,past_2020,'2020',col_index,col_need,2)

In [ ]:
sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

sum_df=sum_df.fillna(0)

sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

sum_df=sum_df.loc[ ~(sum_df == 0).all(axis=1),:]

In [28]:
sum_df.sort_values(col_index).to_excel('past_data_2000till2020_muni.xlsx')

## השוואות בין התרחישים

In [29]:
jtmt_full=jtmt
jtmt=jtmt.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

### אוכלוסייה  לפי מחוז תרחיש כל עשור

In [ ]:
col_index=['zonetype','Muni_Heb']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2050']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

In [31]:
sum_df.to_excel('sum_scenario_pop_by_muni_for_pre.xlsx')

In [ ]:
col_index=['zonetype']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2050','total_emp_2050']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

In [ ]:
col_index=['jeru_metro']

col_need=[
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050','total_emp_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

# List of index values to keep
index_values_to_keep = ['Jerusalem', 'Judea and Samaria', 1]

sum_df=sum_df.loc[index_values_to_keep].sort_values(by='scenario')

In [34]:
col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2050']

In [35]:
sum_df[col_need].to_excel('sum_scenario_pop_by_macoz_for_pre.xlsx')

In [36]:
col=['scenario','pop_without_dorms_yeshiva_2050', 'total_emp_2050']
df=sum_df[col].reset_index()
df['index']=df['index'].astype(str)
df.sort_values(by='index').to_excel('sum_scenario_machoz_pop_emp.xlsx')

העיבוד הבא צריך להכין איפשהו מחדש


sum_df[['scenario',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']].sort_values('group').to_excel('sum_scenario_group_pop.xlsx')

### אוכלוסייה ותעסוקה לפי מחוז תרחיש בשנת 2050

In [ ]:
col_index=['zonetype']
col_need=['pop_without_dorms_yeshiva','total_emp']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'2020',col_index,col_need,-3)

In [ ]:
col_index=['jeru_metro']
sum_df=sum_scenarios(sum_df,jtmt,'2020',col_index,col_need,-3)

In [39]:
# List of index values to keep
index_values_to_keep = ['Jerusalem', 'Judea and Samaria', 1]

sum_df=sum_df.loc[index_values_to_keep].sort_values(by='scenario').to_excel('kayim_sum_machoz_pop_emp.xlsx')


col=['zonetype']

col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(jtmt,index='zonetype',aggfunc=sum)[col_need],-4)

df['year']=2050

df['scenario']='jtmt'
sum_df=df
col=['zonetype']

col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(iplan,index='zonetype',aggfunc=sum)[col_need],-4)

df['year']=2050

df['scenario']='iplan'
sum_df=pd.concat([sum_df,df],axis=0)
col=['zonetype']

col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(bau,index='zonetype',aggfunc=sum)[col_need],-4)

df['year']=2050

df['scenario']='bau'
sum_df=pd.concat([sum_df,df],axis=0)
sum_df=sum_df.sort_values('zonetype')
col=['scenario','pop_without_dorms_yeshiva_2050', 'total_emp_2050']
sum_df[col].to_excel('sum_scenario_machoz_pop_emp.xlsx')


### אוכלוסייה  לפי מגזר חומשים ותרחיש

In [40]:
bau_full=bau
bau=bau.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

In [41]:
iplan_full=iplan
iplan=iplan.query('zonetype=="Jerusalem" | zonetype=="Judea and Samaria" ')

In [ ]:
iplan=group_sector(iplan)

In [ ]:
bau=group_sector(bau)

In [ ]:
jtmt=group_sector(jtmt)

In [ ]:
col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']

sum_df = pd.DataFrame()

sum_df=sum_scenarios(sum_df,jtmt,'jtmt','group',col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan','group',col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau','group',col_need,-3)

In [46]:
sum_df[['scenario',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']].sort_values('group').to_excel('sum_scenario_group_pop.xlsx')

### אוכלוסיה לפי תרחיש ורשות

In [ ]:
col_need=['pop_without_dorms_yeshiva_2050','total_emp_2050']

df=round(pd.pivot_table(jtmt,index=['zonetype','Muni_Heb'],aggfunc=sum)[col_need],-2)

df['scenario']='jtmt'

sum_df=df

In [48]:
col_index=['zonetype','Muni_Heb']

In [ ]:
sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-2)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-2)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva','total_emp'],-2)

In [ ]:
sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

In [51]:
sum_df.to_excel('sum_scenario_Muni_Heb_pop_emp.xlsx')

### אוכלוסייה לפי מחוז מגזר חומשים ותרחיש

In [ ]:
sum_df = pd.DataFrame()

col_index=['zonetype','group']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

In [ ]:
col_index=['jeru_metro','group']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

In [54]:
sum_df.index.names = ['zonetype', 'group']
sum_df.reset_index()[['zonetype','group','scenario',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050']].sort_values(by=['zonetype','group']).query('zonetype!=0').to_excel('sum_scenario_machoz_group_pop.xlsx')

### אוכלוסייה בשכונות של ירושלים לפי מגזר ותרחיש

In [ ]:
sum_df = pd.DataFrame()

col_index=['SCHN_NAME','group']

col_need=[ 'pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

In [ ]:
sum_df=sum_df.pivot_table(index=col_index,columns='scenario',aggfunc=sum)

sum_df=sum_df.fillna(0)

sum_df = sum_df.loc[:, ~(sum_df == 0).all(axis=0)]

sum_df=sum_df.loc[ ~(sum_df == 0).all(axis=1),:]

In [57]:
sum_df.to_excel('sum_scenario_jeru_group_pop.xlsx')

### אוכלוסייה  לפי מגזר ותרחיש השוואה לפי מחוז

In [ ]:
sum_df = pd.DataFrame()

col_index=['zonetype','group']

col_need=['pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

In [ ]:
col_index=['jeru_metro','group']

col_need=['pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

In [60]:
sum_df.index.names = ['zonetype', 'group']

In [ ]:
compare_scenarios(sum_df,['zonetype', 'group']).T.to_excel('sum_scenario_machoz_group_pop_for_precent.xlsx')

### אוכלוסייה  לפי מגזר ותרחיש השוואה לפי העיר ירושלים

In [ ]:
sum_df = pd.DataFrame()

col_index=['in_jerusal','group']

col_need=['pop_without_dorms_yeshiva_2050']

sum_df=sum_scenarios(sum_df,jtmt,'jtmt',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,iplan,'iplan',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,bau,'bau',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,jtmt,'kayim',col_index,['pop_without_dorms_yeshiva'],-3)

compare_scenarios(sum_df,col_index).T.to_excel('sum_scenario_jeru_group_pop_for_precent.xlsx')

## השוואות בין הגירסאות

### אוכלוסיה חינוך ותעסוקה סך הכל 

In [63]:
columns_to_rename = {
    'univ': 'uni_students',
    'univ_2040': 'uni_students_2040',
    'student_yeshiva_and_kollim': 'student_yeshiva',
    'student_yeshiva_and_kollim_2040': 'student_yeshiva_2040'
}

# Rename the columns
jtmt_full.rename(columns=columns_to_rename, inplace=True)

In [ ]:
sum_df = pd.DataFrame()

col_index=['zonetype']

col_need=['pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040','student','student_2040','uni_students','uni_students_2040','student_yeshiva','student_yeshiva_2040']

sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

col_rel=['zonetype','pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040','student','student_2040','uni_students','uni_students_2040','student_yeshiva','student_yeshiva_2040']

sum_df=sum_scenarios(sum_df,v3[col_rel],'v3',col_index,col_need,-3)

In [65]:
sum_df.groupby(by='scenario').sum().T.to_excel('versions_total_pop_emp_student.xlsx')

### אוכלוסיה ותעסוקה לפי מחוז 

In [66]:
v3['jeru_metro']=0
v3.loc[v3['in_jerusalem_metropolin']=='yes','jeru_metro']=1

In [ ]:
sum_df = pd.DataFrame()

col_index=['zonetype']

col_need=['pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040']

all_col=['zonetype','pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040']

sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3[all_col],'v3',col_index,col_need,-3)

In [ ]:
col_index=['jeru_metro']

col_need=['pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040']

all_col=['jeru_metro','pop_without_dorms_yeshiva','pop_without_dorms_yeshiva_2040','total_emp','total_emp_2040']

sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3[all_col],'v3',col_index,col_need,-3)

In [69]:
sum_df.index.names = ['zonetype']

In [ ]:
compare_scenarios(sum_df,['zonetype']).query('zonetype!=0').to_excel('versions_machoz_pop_emp.xlsx')

עד כאן הגעתי

###   חומש אוכלוסייה לפי מגזר 

In [71]:
jtmt_full=group_sector(jtmt_full)

v3=group_sector(v3)

In [ ]:
sum_df = pd.DataFrame()

col_index=['group']

col_need=['pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040']

sum_df=sum_scenarios(sum_df,jtmt_full,'v4',col_index,col_need,-3)

col_rel=['group','pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040']

sum_df=sum_scenarios(sum_df,v3[col_rel],'v3',col_index,col_need,-3)

sum_df.sort_values('group').to_excel('sum_versions_group_pop.xlsx')

###   התפלגות גילים 

In [73]:
v3_age=up_load_df(save_file,'220303_התפלגות_גילים_V3_3_Published')

v3=v3.merge(v3_age,right_on='taz_V3_2',left_on='Taz_num',how='left')

In [74]:
def sum_age(df,name_col,col_to_sum):
    df[name_col]=df[col].sum(axis=1)
    return df

In [75]:
col=['2020_0-4',
 '2020_5-9',
 '2020_10-14',
 '2020_15-19']

v3=sum_age(v3,'pop_0_20_2020',col)

col=[
 '2020_20-24',
 '2020_25-29',
 '2020_30-34',
 '2020_35-39',
 '2020_40-44']

v3=sum_age(v3,'pop_21_45_2020',col)

col=[
 '2020_45-49',
 '2020_50-54',
 '2020_55-59',
 '2020_60-64']

v3=sum_age(v3,'pop_45_65_2020',col)

col=[
 '2020_65-69',
 '2020_70-74',
 '2020_75+']

v3=sum_age(v3,'pop_65+_2020',col)

In [76]:
col=['2040_0-4',
 '2040_5-9',
 '2040_10-14',
 '2040_15-19']

v3=sum_age(v3,'pop_0_20_2040',col)

col=[
 '2040_20-24',
 '2040_25-29',
 '2040_30-34',
 '2040_35-39',
 '2040_40-44']

v3=sum_age(v3,'pop_21_45_2040',col)

col=[
 '2040_45-49',
 '2040_50-54',
 '2040_55-59',
 '2040_60-64']

v3=sum_age(v3,'pop_45_65_2040',col)

col=[
 '2040_65-69',
 '2040_70-74',
 '2040_75+']

v3=sum_age(v3,'pop_65+_2040',col)

In [77]:
percentage_cols=['pop_0_20_2040','pop_21_45_2040','pop_45_65_2040','pop_65+_2040']

In [78]:
v3[percentage_cols] = v3[percentage_cols].mul(v3['pop_without_dorms_yeshiva_2040'], axis=0)

In [79]:
percentage_cols=['pop_0_20_2020','pop_21_45_2020','pop_45_65_2020','pop_65+_2020']

In [80]:
v3[percentage_cols] = v3[percentage_cols].mul(v3['pop_without_dorms_yeshiva'], axis=0)

In [ ]:
BaseProjections_version_date = get_newest_date_file(r'{}\JTMT'.format(forecast_version_folder_location), 'BaseProjections2040_')
BaseProjections_version_date

In [82]:
jtmt_2040=up_load_df(r'{}\JTMT'.format(forecast_version_folder_location),r'BaseProjections2040_{}_jtmt'.format(BaseProjections_version_date))

jtmt_2040=jtmt_full.merge(jtmt_2040,right_on='TAZ',left_on='Taz_num',how='left')

col=['age0_4',
 'age5_9',
 'age10_14',
 'age15_19',]

jtmt_2040=sum_age(jtmt_2040,'pop_0_20_2040',col)

col=['age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',]

jtmt_2040=sum_age(jtmt_2040,'pop_21_45_2040',col)

col=[
'age45_49',
 'age50_54',
 'age55_59',
 'age60_64',]

jtmt_2040=sum_age(jtmt_2040,'pop_45_65_2040',col)

col=[
  
 'age65_69',
 'age70_74',
 'age75up',]

jtmt_2040=sum_age(jtmt_2040,'pop_65+_2040',col)

In [83]:
jtmt_2020=up_load_df(forecast_version_base_year_folder_location,r'BaseProjections2020_{}'.format(BaseProjections_version_date))

jtmt_2020=jtmt_full.merge(jtmt_2020,right_on='TAZ',left_on='Taz_num',how='left')

col=['age0_4',
 'age5_9',
 'age10_14',
 'age15_19',]

jtmt_2020=sum_age(jtmt_2020,'pop_0_20_2020',col)

col=['age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',]

jtmt_2020=sum_age(jtmt_2020,'pop_21_45_2020',col)

col=[
'age45_49',
 'age50_54',
 'age55_59',
 'age60_64',]

jtmt_2020=sum_age(jtmt_2020,'pop_45_65_2020',col)

col=[
  
 'age65_69',
 'age70_74',
 'age75up',]

jtmt_2020=sum_age(jtmt_2020,'pop_65+_2020',col)

In [84]:
jtmt_2020=group_sector(jtmt_2020)

In [85]:
jtmt_2040=group_sector(jtmt_2040)

In [ ]:
sum_df = pd.DataFrame()

col_index=['group']

col_need=['pop_0_20_2020',
 'pop_21_45_2020',
 'pop_45_65_2020',
 'pop_65+_2020']

combined_columns = col_index + col_need

sum_df=sum_scenarios(sum_df,jtmt_2020,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3[combined_columns],'v3',col_index,col_need,-3)

In [ ]:
col_need=[
 'pop_0_20_2040',
 'pop_21_45_2040',
 'pop_45_65_2040',
 'pop_65+_2040']

combined_columns = col_index + col_need

sum_df=sum_scenarios(sum_df,jtmt_2040,'v4',col_index,col_need,-3)

sum_df=sum_scenarios(sum_df,v3[combined_columns],'v3',col_index,col_need,-3)

In [88]:
sum_df.groupby(by=['group','scenario']).sum().to_excel('sum_versions_group_age_des.xlsx')